In [ ]:
from gettext import install
import pip


pip install rasterio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install arcpy

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement arcpy (from versions: none)
ERROR: No matching distribution found for arcpy


In [ ]:
import os
import re
import arcpy
from datetime import datetime

# CSV 파일 경로 설정
csv_file = r"E:\2025_Jeju_SectorCoupling\2024_Jeju_Solar.csv"
hillshade_dirs = {
    '제주': r"E:\2025_Jeju_SectorCoupling\SolarArea_Jeju",
    '고산': r"E:\2025_Jeju_SectorCoupling\SolarArea_Gosan",
    '서귀포': r"E:\2025_Jeju_SectorCoupling\SolarArea_Seoguipo"
}
output_dirs = {
    '제주': r"E:\2025_Jeju_SectorCoupling\PVgen_Jeju",
    '고산': r"E:\2025_Jeju_SectorCoupling\PVgen_Gosan",
    '서귀포': r"E:\2025_Jeju_SectorCoupling\PVgen_Seoguipo"
}

# 파일명에서 datetime 추출 함수
def extract_datetime(filename):
    match = re.search(r'(\d{4}-\d{2}-\d{2} \d{2})-\d{2}-\d{2}\+\d{2}-\d{2}', filename)
    return datetime.strptime(match.group(1), '%Y-%m-%d %H:%M') if match else None

# CSV 파일을 ArcPy로 로드 (CSV를 테이블로 읽음)
table = arcpy.da.TableToNumPyArray(csv_file, ['지점명', '일시', '일사(MJ/m2)'])

# 각 행을 반복 처리
for row in table:
    location, timestamp = row['지점명'], row['일시']
    if location not in hillshade_dirs:
        continue
    
    # 해당 지점의 Hillshade 파일 가져오기
    raster_files = arcpy.ListFiles(os.path.join(hillshade_dirs[location], '*.tif'))
    
    for raster_file in raster_files:
        raster_time = extract_datetime(raster_file)
        if raster_time != timestamp:
            continue
        
        # Hillshade 래스터 열기 (ArcPy로 래스터 처리)
        raster = arcpy.Raster(raster_file)
        cell_area = raster.meanCellHeight * raster.meanCellWidth
        
        # 일사량(MJ/m2)을 'irradiance' 변수에 반영
        irradiance = row['일사(MJ/m2)']
        
        # Hillshade가 null인 경우 결과도 null로 설정
        hillshade_factor = arcpy.sa.Con(arcpy.sa.IsNull(raster), None, arcpy.sa.Con(raster == 0, 0, 1))  
        # Con(Null일 경우 None, Hillshade가 0이면 0, 그렇지 않으면 1)
        
        # 발전량 계산 (None 값에 대해 NaN을 설정하여 계산 오류 방지)
        solar_generation = arcpy.sa.Con(hillshade_factor != None, irradiance * 0.22 * cell_area * hillshade_factor, None)
        
        # 결과를 새로운 래스터 파일로 저장
        output_filename = f"PVgen_{location}_{timestamp.strftime('%Y-%m-%d %H-%M')}.tif"
        output_path = os.path.join(output_dirs[location], output_filename)
        
        # ArcPy로 래스터 결과 저장
        solar_generation.save(output_path)
        
        # 파일이 저장되었음을 알림
        print(f"파일이 저장되었습니다: {output_path}")
